In [1]:
# import packages
import cv2
import dlib
import numpy as np
from PIL import Image, ImageSequence

!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2
datFile =  "/content/shape_predictor_68_face_landmarks.dat"

--2023-04-06 01:20:34--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M   105MB/s    in 0.6s    

2023-04-06 01:20:34 (105 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [3]:
# get sample GIF files
!wget --no-check-certificate \
  https://media.tenor.com/ei_IhOGEk_YAAAAd/shinyeeun-the-glory.gif \
  -O /content/shinyeeun-the-glory.gif
!wget --no-check-certificate \
  https://media.tenor.com/YZuvasRsENIAAAAd/angelina-jolie-laugh.gif \
  -O /content/three-people.gif

--2023-04-06 01:20:50--  https://media.tenor.com/ei_IhOGEk_YAAAAd/shinyeeun-the-glory.gif
Resolving media.tenor.com (media.tenor.com)... 172.253.122.95, 172.253.63.95, 142.251.16.95, ...
Connecting to media.tenor.com (media.tenor.com)|172.253.122.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658606 (643K) [image/gif]
Saving to: ‘/content/shinyeeun-the-glory.gif’

/content/shinyeeun- 100%[===================>] 643.17K  --.-KB/s    in 0.007s  

2023-04-06 01:20:50 (94.1 MB/s) - ‘/content/shinyeeun-the-glory.gif’ saved [658606/658606]

--2023-04-06 01:20:50--  https://media.tenor.com/YZuvasRsENIAAAAd/angelina-jolie-laugh.gif
Resolving media.tenor.com (media.tenor.com)... 172.253.122.95, 172.253.63.95, 142.251.16.95, ...
Connecting to media.tenor.com (media.tenor.com)|172.253.122.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3178376 (3.0M) [image/gif]
Saving to: ‘/content/three-people.gif’

/content/three-peop 100%[=============

In [4]:
# Load the image sequence using PIL
# gif영상 내 각 frame을 list화하여 저장
def load_gif(file_path):
  gif = Image.open(file_path)
  frames = [frame.convert('RGB').copy() for frame in ImageSequence.Iterator(gif)]
  return frames

dlib 라이브러리를 이용하여 image sequence 내 각 이미지에서 얼굴 랜드마크 검출하는 함수 작성
- 랜드마크는 OpenCV 함수 내 cv2.circle 함수를 이용하여 점 찍어주기기

In [5]:
# Write a function for detecting facial landmarks in an image sequence using the dlib library
def detect_landmarks_dlib(frames):
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

  landmarks_frames = [] # List where images with face landmarks are saved # 복사된 이미지 안에 68개 랜드마크 찍기

  # Write the code below
  det_faces = dlib.full_object_detections()
  
  for i, frame in enumerate(frames):      # 각 이미지에 대해 얼굴 탐지
    # detect faces in img using dlib frontal face detector
    frame_ndarray = np.array(frame)
    dets = detector(frame_ndarray, 1)
    # print coordiates of detected face
    for i, d in enumerate(dets): 
      f = predictor(frame_ndarray, d)
      det_faces.append(f)

  ##############

  for i, frame in enumerate(frames):
    frame_ndarray = np.array(frame)
    # make a copy of the input image
    #frame_cpy = frame.copy()
    # get i_th face information
    f = det_faces[i]
    for j in range(f.num_parts):
        cv2.circle(frame_ndarray, (f.part(j).x,f.part(j).y), 2, (0, 0, 255), -1)
    # save image with overlay
    landmarks_frames.append(frame_ndarray)
    # dlib.save_image(frame_cpy, f'frame_{i}.png');  
  
  return landmarks_frames


image sequence를 입력받아서 OpenCV 라이브러리를 이용하여 .mp4 포맷의 비디오로 저장하는 함수 작성



In [6]:
# 프레임 > 비디오 포맷으로 저장하는 함수 구현
# frame 'image sequence'이므로 PIL image의 배열일 것이다.
def save_video(frames, file_name):
  frame_array = []
  ####추가
  fps = 30
  for img in frames:
    # ((참고)) pil이미지의 .size함수의 반환값은 (width, height), ndarray의 .shape는 (height, width, color)순으로 반환한다.
    frame_array.append(img)
  # cv2.VideoWriter함수 이용하여 가상의 비디오 out 형성 // cv2.VideoWriter(filename, fourcc, fps, frameSize, isColor=None)
  out = cv2.VideoWriter(file_name, cv2.VideoWriter_fourcc(*'DIVX'), fps, img.size, True)
  # frames의 이미지를 비디오로 조합
  for img in frames:
    # OpenCV는 BRG 색상 형식, matplotlib는 RGB 색상 형식을 사용하므로, cv2.cvtColor
    out.write(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
  out.release()

위에서 작성한 함수들을 활용하여 (1) GIF 파일을 읽고 (2) 얼굴 랜드마크 검출하고 (3) 동영상으로 저장 하는 코드 작성



In [7]:
# (1) Load the gif file using the PIL library
file_path = '/content/shinyeeun-the-glory.gif'
#file_path = '/content/three-people.gif'
# 동영상 파일명
file_name = '/content/shinyeeun-the-glory-video.mp4'
#file_name = '/content/three-people-video.mp4'
frames = load_gif(file_path)

# (2) Use the dlib library for detecting facial landmarks in the image sequence
landmarks_frames = detect_landmarks_dlib(frames)
# 점이 잘 찍혔는지 한 프레임만 출력해서 확인
#pil_image=Image.fromarray(landmarks_frames[1]) 
#pil_image.show()

# (3) Save the landmark-detected image sequence as a mp4 video using the OpenCV library
pil_landmarks_frames = []
for i in range(len(landmarks_frames)):
  pil_image=Image.fromarray(landmarks_frames[i])
  pil_landmarks_frames.append(pil_image)  # 68 landmarks 찍힌 프레임들 조합해 pil이미지 시퀀스로 만들기
save_video(pil_landmarks_frames, file_name)


얼굴 랜드마크 검출하는 함수를 mediapipe 라이브러리를 이용해서 해보자.

In [8]:
# install mediapipe library
!pip install mediapipe
import mediapipe as mp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 34.4 MB/s eta 0:00:00


mediapipe 라이브러리 
https://google.github.io/mediapipe/solutions/face_mesh.html

mediapipe 라이브러리를 이용하여 image sequence 내 각 이미지에서 얼굴 메쉬(face mesh)를 검출하는 함수 작성
- face mesh 검출에는 mp.solutions.face_mesh 활용
- 검출된 mesh를 그릴 때는 mp.solutions.drawing_utils 활용

In [9]:
# Write a function that detects face meshes in a sequence of images using the mediapipe library
def detect_face_mesh(frames):
  mp_face_mesh = mp.solutions.face_mesh #얼굴검출 모듈
  mp_drawing = mp.solutions.drawing_utils # 얼굴 위에 유틸 그리는 모듈

  face_mesh_frames = [] # List where images with face landmarks are saved

  # Write the code below
  # refine_landmarks < True면 눈, 코, 입술의 검출이 정교해짐
  # static_image < True면 영상의 매 프레임마다 각각, False면 첫 프레임 얼굴검출 data로부터 뒤 프레임은 Tracking을 하여 얼굴검출함
  # max_num < 최대로 검출할 얼굴의 개수
  face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, static_image_mode=True, max_num_faces=3, )
  drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

  for image in frames:
    # 얼굴 검출
    img_np = np.array(image)
    results = face_mesh.process(img_np)

    # Face Mesh 그리기
    for single_face_landmarks in results.multi_face_landmarks:
      mp_drawing.draw_landmarks(
          image=img_np,
          landmark_list=single_face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=drawing_spec,
          connection_drawing_spec=drawing_spec,
      )
    #이미지로 저장
    pil_image=Image.fromarray(img_np)
    face_mesh_frames.append(pil_image)

  return face_mesh_frames

위에서 작성한 함수들을 활용하여 (1) GIF 파일을 읽고 (2) mediapipe 라이브러리로 face mesh 검출하고 (3) 동영상으로 저장 하는 코드 작성

In [10]:
# (1) Load the gif file using the PIL library
file_path = '/content/shinyeeun-the-glory.gif'
#file_path = '/content/three-people.gif'
# 동영상 파일명
file_name = '/content/shinyeeun-the-glory-mp.mp4'
#file_name = '/content/three-people-mp.mp4'
frames = load_gif(file_path)

# (2) Detect face meshes for all images in the loaded gif file using the mediapipe library
pil_landmarks_frames = detect_face_mesh(frames)

# (3) Save the face mesh detected image sequence as a mp4 video using the OpenCV library
save_video(pil_landmarks_frames, file_name)


(+추가) 위 코드 작성을 모두 완료했다면, 지난 시간에 실습했던 alignment 코드를 활용해서, 본인의 얼굴에서 landmark를 검출한 후 alignment 시켜보세요!

참고한 코드 출처:

(1) [mediapipe - Face Mesh](https://denev6.tistory.com/entry/Face-Mesh) 

(2) [[파이썬] MediaPipe 얼굴 그물망(Face Mesh)](https://puleugo.tistory.com/5)

(3) [[ Python ] 이미지들을 동영상으로 만들기 (images -> mp4)](https://data-newbie.tistory.com/384)